# Introduction
One of my favourite features in Spotify is the concept the Daily Mixes. 
Right now Spotify keeps 6 daily mixes for me, each with it's own kind of music. 
For me this is amazing, as I tend to listen to music from very varied genres but might not always want to mix them.

I thought it would be fun to learn about clustering methods by trying to cluster some tracks extracted from these mixes. 
Spotify does a great job of playing tracks that I think belong to the same class of songs, let's see if I can do the same!

Using some of the functions from my module `happify` I will extract song from playlists I create in Spotify, together with some metadata such as:
* Key
* Time signature
* BPM
* Mode (Major/Minor)
* Other spotify track features:
     * Dancability
     * Valence (Positiveness)
     * etc.
     
Find the full list of features available [here]("https://beta.developer.spotify.com/documentation/web-api/reference/tracks/get-audio-features/").

In [67]:
import happify
import imp

imp.reload(happify)

<module 'happify' from 'C:\\Users\\Gustav\\Data Science\\Happify\\happify.py'>

Short term authentication.

In [ ]:
access_token = happify.authorize()

Fetch playlist metadata for the current user to find the playlist id of the playlist you want to fetch data for.

In [20]:
playlist_df = happify.get_playlists(access_token)

I have prepared a playlist called `Swedsish Pop and American Hip Hop` which is a mix of two of my daily mixes. One with American Hip Hop and one with Swedish Pop. I will try to apply some clustering to it.

In [24]:
playlist_id = '5DI5cgV87pERI0aagbGqup'

In [19]:
user_id = happify.get_user_id(access_token)

Extract tracks from the playlist.

In [42]:
tracks = happify.get_playlist_tracks(user_id, playlist_id, access_token)

In [47]:
print("Currently {} tracks in the list".format(len(tracks)))

Currently 51 tracks in the list


Extract some extra features for all tracks.

In [52]:
track_feature_df = happify.track_features(access_token, tracks)

In [53]:
track_feature_df.head()

,album,artists,id,name,acousticness,analysis_url,danceability,duration_ms,energy,instrumentalness,...,liveness,loudness,mode,speechiness,tempo,time_signature,track_href,type,uri,valence
0,{'name': 'Snart Skiner Poseidon'},[{'external_urls': {'spotify': 'https://open.s...,2eljaMe4nUwA2vqdzVV8FX,Snart Skiner Poseidon,0.828000,https://api.spotify.com/v1/audio-analysis/2elj...,0.253,178978,0.482,0.000004,...,0.1530,-5.220,1,0.0306,140.507,3,https://api.spotify.com/v1/tracks/2eljaMe4nUwA...,audio_features,spotify:track:2eljaMe4nUwA2vqdzVV8FX,0.3490
1,"{'name': 'Svart, vitt och allt däremellan'}",[{'external_urls': {'spotify': 'https://open.s...,41L7xlhy4S63YFzBhJOlNU,Vitt,0.485000,https://api.spotify.com/v1/audio-analysis/41L7...,0.561,280653,0.666,0.004750,...,0.1270,-7.879,1,0.0376,109.981,4,https://api.spotify.com/v1/tracks/41L7xlhy4S63...,audio_features,spotify:track:41L7xlhy4S63YFzBhJOlNU,0.3540
2,{'name': 'Kom hem som nån annan'},[{'external_urls': {'spotify': 'https://open.s...,2Q8yH8RdEMLR56z9seHTjT,Längs hörnen,0.853000,https://api.spotify.com/v1/audio-analysis/2Q8y...,0.400,260600,0.371,0.000686,...,0.3280,-11.405,0,0.0387,157.795,4,https://api.spotify.com/v1/tracks/2Q8yH8RdEMLR...,audio_features,spotify:track:2Q8yH8RdEMLR56z9seHTjT,0.0899
3,{'name': 'Pantamera'},[{'external_urls': {'spotify': 'https://open.s...,7I6yrk52OAtrGhjFhNhtrt,Pantamera,0.000836,https://api.spotify.com/v1/audio-analysis/7I6y...,0.674,135000,0.865,0.002720,...,0.1650,-6.496,1,0.0464,128.046,4,https://api.spotify.com/v1/tracks/7I6yrk52OAtr...,audio_features,spotify:track:7I6yrk52OAtrGhjFhNhtrt,0.8260
4,{'name': '...And Then There Was Timo'},[{'external_urls': {'spotify': 'https://open.s...,4mdnleRe8GwP81gGWjolU6,Creep,0.475000,https://api.spotify.com/v1/audio-analysis/4mdn...,0.351,194400,0.328,0.000000,...,0.0963,-7.606,1,0.0335,104.426,1,https://api.spotify.com/v1/tracks/4mdnleRe8GwP...,audio_features,spotify:track:4mdnleRe8GwP81gGWjolU6,0.0742


Let's clean up and drop some unecessary columns.

In [54]:
track_feature_df.columns

Index(['album', 'artists', 'id', 'name', 'acousticness', 'analysis_url',
       'danceability', 'duration_ms', 'energy', 'instrumentalness', 'key',
       'liveness', 'loudness', 'mode', 'speechiness', 'tempo',
       'time_signature', 'track_href', 'type', 'uri', 'valence'],
      dtype='object')

In [56]:
track_feature_df['artists'] = track_feature_df['artists'].apply(lambda x: [artist['name'] for artist in x])

track_feature_df['album'] = track_feature_df['album'].apply(lambda x: x['name'])

In [60]:
track_feature_df = track_feature_df.drop(['id', 'analysis_url', 'track_href', 'uri', 'type'], 1)

In [65]:
track_feature_df

,album,artists,name,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
0,Snart Skiner Poseidon,[Joel Alme],Snart Skiner Poseidon,0.828000,0.253,178978,0.482,0.000004,2,0.1530,-5.220,1,0.0306,140.507,3,0.3490
1,"Svart, vitt och allt däremellan",[Daniel Adams-Ray],Vitt,0.485000,0.561,280653,0.666,0.004750,7,0.1270,-7.879,1,0.0376,109.981,4,0.3540
2,Kom hem som nån annan,[Emil Jensen],Längs hörnen,0.853000,0.400,260600,0.371,0.000686,0,0.3280,-11.405,0,0.0387,157.795,4,0.0899
3,Pantamera,[Familjen],Pantamera,0.000836,0.674,135000,0.865,0.002720,0,0.1650,-6.496,1,0.0464,128.046,4,0.8260
4,...And Then There Was Timo,[Timo Räisänen],Creep,0.475000,0.351,194400,0.328,0.000000,7,0.0963,-7.606,1,0.0335,104.426,1,0.0742
5,Prinsen av Peking,[Markus Krunegård],New York,0.006240,0.336,269413,0.827,0.000004,2,0.0753,-5.506,1,0.0290,158.830,4,0.4410
6,Det kommer aldrig va över för mig,[Håkan Hellström],Du kan gå din egen väg,0.214000,0.704,239133,0.759,0.001440,9,0.0883,-6.581,1,0.0774,140.021,4,0.8130
7,The Storm,[HOFFMAESTRO],The Storm,0.039700,0.525,253200,0.827,0.002690,5,0.0815,-5.503,0,0.0892,155.072,4,0.7630
8,En apa som liknar dig,[Olle Ljungström],Med eller utan,0.049200,0.717,209627,0.475,0.000342,2,0.1100,-9.577,1,0.0255,98.333,4,0.3760
9,Väntar på en Ängel,[Oskar Linnros],Oavsett,0.104000,0.602,240493,0.668,0.057900,8,0.2750,-9.322,1,0.0538,154.977,4,0.5440


Cool, we have some data to analyse!

# Exploratory Data Analysis
Let's start of by exploring the features!